In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
df = pd.read_excel("../datasets/butecos_2025_geolocated.xlsx")

In [3]:
df["Detalhamento Prato"] = None
df["Horario"] = None
df["Telefone"] = None

for i, row in tqdm(df.iterrows(), total=len(df)):
    url_detalhes = row["Detalhes"]
    response = requests.get(url_detalhes)
    soup = BeautifulSoup(response.content, "html.parser")
    
    info = soup.find('div', class_='section-text').find_all('p')
    
    try:
        ########## detalhamento do prato ##########
        detalhamento_prato = info[0]

        # Pega o texto do <b>
        bold_text = detalhamento_prato.b.get_text()

        # Pega o texto completo e remove a parte em <b>
        text_after_b = detalhamento_prato.get_text().replace(bold_text, '', 1).strip()
        
        ########## telefone ##########
        telefone = info[-2].text.strip()
        
        ########### horario ##########
        horario = info[-1].text.strip()
        
        # adiciona ao dataframe
        df.at[i, "Detalhamento Prato"] = text_after_b
        df.at[i, "Telefone"] = telefone
        df.at[i, "Horario"] = horario
    
    except:
        # Se não encontrar o telefone ou horario, continua
        continue

100%|██████████| 1193/1193 [12:50<00:00,  1.55it/s]


In [4]:
df.to_excel("../datasets/butecos_2025_detailed.xlsx", index=False)